In [7]:
homographs = []
with open("homographs_id.dict", "w") as w:
    with open("homographs_id.tsv", "r") as f:
        for line in f:
            row = line.strip().split("\t")
            w.write(f"{row[0]}\t{row[1]}\n")
            w.write(f"{row[0]}\t{row[2]}\n")
            
            homographs.append({"word": row[0], "phonemes": row[1], "roles": row[3]})
            homographs.append({"word": row[0], "phonemes": row[2], "roles": row[4]})


In [11]:
import pandas as pd

with open("homographs_id.corpus", "r") as f:
     g2p_alignments = [line.strip() for line in f]

for entry, g2p_alignment in zip(homographs, g2p_alignments):
    entry["g2p_alignment"] = g2p_alignment

df = pd.DataFrame(homographs)
df.to_csv("homographs_id.csv", index=False)

In [ ]:
alignments = []
with open("lexicon_id.corpus", "r") as f:
    for line in f:
        alignments.append(line.strip())
        
print(alignments[:5])

In [6]:
import pandas as pd
df1 = pd.read_csv("lexicon_id.csv")
df2 = pd.read_csv("homographs_id.csv")


df_combined = pd.concat([df1, df2], ignore_index=True)
df_combined = df_combined.fillna("")
df_combined['pron_order'] = df_combined.groupby('word').cumcount()
df_combined

,word,phonemes,g2p_alignment,roles,pron_order
0,a,a,a}a,,0
1,b,b e,b}b|e,,0
2,c,tʃ e,c}tʃ|e,,0
3,d,d e,d}d|e,,0
4,e,e,e}e,,0
...,...,...,...,...,...
27777,tepok,t e p o ʔ,t}t e}e p}p o}o k}ʔ,A,1
27778,terapi,t e r a p i,t}t e}e r}r a}a p}p i}i,N,0
27779,terapi,t ə r a p i,t}t e}ə r}r a}a p}p i}i,A,1
27780,teras,t e r a s,t}t e}e r}r a}a s}s,N,0


In [9]:
import pandas as pd
import sqlite3
import pandas as pd


# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('id_lexicon.db')
cursor = conn.cursor()

# Create the table with an id column and a roles column
cursor.execute('''
CREATE TABLE IF NOT EXISTS id_phonemes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    word TEXT,
    phonemes TEXT,
    pron_order INTEGER,
    g2p_alignment TEXT,
    role TEXT DEFAULT ''
)
''')

# Insert data from the DataFrame into the SQLite table
for index, row in df_combined.iterrows():
    cursor.execute('''
    INSERT INTO id_phonemes (word, phonemes, pron_order, g2p_alignment, role)
    VALUES (?, ?, ?, ?, ?)
    ''', (row['word'], row['phonemes'], row["pron_order"], row['g2p_alignment'], row['roles']))

# Commit the changes and close the connection
conn.commit()
conn.close()    
